<a href="https://colab.research.google.com/github/tae-min-lee/DL_jupyter/blob/master/train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 훈련

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import numpy as np
from keras.utils import np_utils

### 데이터 전리부분


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/heart stroke/prepro_data_2.csv')

In [ ]:
data.head(2)

,Unnamed: 0,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,stroke
0,0,30669,Male,3.0,0,0,No,children,Rural,never smoked,0
1,1,30468,Male,58.0,1,0,Yes,Private,Urban,never smoked,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43389 entries, 0 to 43388
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      43389 non-null  int64  
 1   id              43389 non-null  int64  
 2   gender          43389 non-null  object 
 3   age             43389 non-null  float64
 4   hypertension    43389 non-null  int64  
 5   heart_disease   43389 non-null  int64  
 6   ever_married    43389 non-null  object 
 7   work_type       43389 non-null  object 
 8   Residence_type  43389 non-null  object 
 9   smoking_status  43389 non-null  object 
 10  stroke          43389 non-null  int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 3.6+ MB


#### 불필요한 컬럼 제거
id값과 Unnamed는 예측에 전혀 필요가 없으니 제거 한다

In [ ]:
h_data = data.drop(['Unnamed: 0','id'],axis=1)

In [ ]:
h_data.head(2)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,stroke
0,Male,3.0,0,0,No,children,Rural,never smoked,0
1,Male,58.0,1,0,Yes,Private,Urban,never smoked,0


In [ ]:
print('성별\n',h_data['gender'].value_counts(),'\n')
print('결혼 여부\n',h_data['ever_married'].value_counts(),'\n')
print('일의 종류\n',h_data['work_type'].value_counts(),'\n')
print('주거 장소\n',h_data['Residence_type'].value_counts(),'\n')
print('흡연여부\n',h_data['smoking_status'].value_counts(),'\n')

성별
 Female    25665
Male      17724
Name: gender, dtype: int64 

결혼 여부
 Yes    27933
No     15456
Name: ever_married, dtype: int64 

일의 종류
 Private          24827
Self-employed     6793
children          6154
Govt_job          5438
Never_worked       177
Name: work_type, dtype: int64 

주거 장소
 Urban    21751
Rural    21638
Name: Residence_type, dtype: int64 

흡연여부
 never smoked       29341
formerly smoked     7487
smokes              6561
Name: smoking_status, dtype: int64 



레이블 값을 분리(stroke)

In [ ]:
data_f = h_data.drop('stroke',axis=1)
data_l = h_data['stroke'] # 0은 정상 1은 심장마비로 사망

범주형 데이터를 매핑을 이용해서 정수형으로 바꿔준다

In [ ]:
gender_mapping = {'Female':0,'Male':1}
married_mapping = {'Yes':0,'No':1}
work_mapping = {'Private':0,'Self-employed':1,'children':2,'Govt_job':3,'Never_worked':4}
Res_mapping = {'Urban':0,'Rural':1}
smoke_mapping ={'never smoked':0,'formerly smoked':1,'smokes':2}

In [ ]:
data_f['gender'] = data_f['gender'].map(gender_mapping)
data_f['ever_married'] = data_f['ever_married'].map(married_mapping)
data_f['work_type'] = data_f['work_type'].map(work_mapping)
data_f['Residence_type'] = data_f['Residence_type'].map(Res_mapping)
data_f['smoking_status'] = data_f['smoking_status'].map(smoke_mapping)

In [ ]:
data_f.info() # 정수형으로 바뀐걸 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43389 entries, 0 to 43388
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gender          43389 non-null  int64  
 1   age             43389 non-null  float64
 2   hypertension    43389 non-null  int64  
 3   heart_disease   43389 non-null  int64  
 4   ever_married    43389 non-null  int64  
 5   work_type       43389 non-null  int64  
 6   Residence_type  43389 non-null  int64  
 7   smoking_status  43389 non-null  int64  
dtypes: float64(1), int64(7)
memory usage: 2.6 MB


In [ ]:
data_f.to_csv('/content/drive/MyDrive/heart stroke/data_f.csv')

#### 정규화
값 정규화를 진행

In [ ]:
feature_data = data_f.to_numpy() # 먼저 넘파이로 바꿔줌
label_data = data_l.to_numpy()

In [ ]:
def Normalize(input_data): #정규화를 시켜주는 부분
  x_data = input_data.astype('float32')

  mean = x_data.mean(axis=0) # 평균값을 구함
  std =x_data.std(axis=0) #표준편차를 구함

  output_data = input_data.astype('float32') # 실수형으로 변환 뒤 모든 데이터에서 평균을 빼주고 표준편차로 나눔
  output_data -= mean
  output_data /= std

  return output_data

In [ ]:
feature_data = Normalize(feature_data)

In [ ]:
feature_data.shape

(43389, 8)

### 정밀도 재현율 부분

In [ ]:
# from keras import backend as K

# def recall_m(y_true, y_pred):  # 재현율을 만드는 함수
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred): # 정밀도 함수
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

### 훈련 검증 테스트 데이터 나누기

훈련과 테스트로 나누고 다음에는 검증으로 한 번 더 나눈다

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_lables, test_labels = train_test_split(feature_data, label_data, test_size=0.2, shuffle=True, stratify=label_data, random_state=34)

In [ ]:
print('테스트 데이터의 수:',len(test_data))

테스트 데이터의 수: 8678


In [ ]:
train_data, valid_data, train_lables, valid_labels = train_test_split(train_data, train_lables, test_size=0.2, shuffle=True, stratify=train_lables, random_state=34)

In [ ]:
print("훈련 데이터:",len(train_data))
print("검증 데이터:",len(valid_data))

훈련 데이터: 27768
검증 데이터: 6943


In [ ]:
from keras import models 
from keras import layers
from keras.layers import Dropout  

model = models.Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(8,)))
model.add(Dropout(0.2))
model.add(layers.Dense(16,activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_1 = model.fit(train_data,
                    train_lables,
                    epochs=3,
                    batch_size=32,
                    validation_data=(valid_data, valid_labels))

Epoch 1/3
868/868 [==============================] - 3s 2ms/step - loss: 0.1178 - accuracy: 0.9745 - val_loss: 0.0809 - val_accuracy: 0.9820
Epoch 2/3
868/868 [==============================] - 2s 2ms/step - loss: 0.0862 - accuracy: 0.9820 - val_loss: 0.0797 - val_accuracy: 0.9820
Epoch 3/3
868/868 [==============================] - 2s 2ms/step - loss: 0.0854 - accuracy: 0.9820 - val_loss: 0.0808 - val_accuracy: 0.9820


In [ ]:
loss, acc  = model.evaluate(test_data,test_labels)

272/272 [==============================] - 0s 1ms/step - loss: 0.0790 - accuracy: 0.9819


In [ ]:
train_lables[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [ ]:
from keras.models import load_model

save_model=model.save("/content/drive/MyDrive/heart stroke/n_2_model.h5")